In [1]:
import pandas as pd
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
import re
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity,euclidean_distances
import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/rkumar-bengaluru/data-science/main/16-Projects/zuel/data/2001_all_materials.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4783 entries, 0 to 4782
Data columns (total 60 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   matnr                       4783 non-null   int64  
 1   zzprncode                   4782 non-null   float64
 2   zzibrnd                     4769 non-null   object 
 3   mtart                       4783 non-null   object 
 4   matkl                       4783 non-null   object 
 5   TherapeuticClass            4783 non-null   object 
 6   PrincipalName               4782 non-null   object 
 7   mstae                       4775 non-null   object 
 8   mstde                       4783 non-null   int64  
 9   mstav                       4762 non-null   object 
 10  mstdv                       4783 non-null   int64  
 11  zzdwrx                      4783 non-null   object 
 12  spart                       4783 non-null   int64  
 13  vtweg                       4783 

In [3]:
df['Description'] = df['Description'].astype(str)
#test = df[df['Description'].str.startswith('DE')]
test = df
#test = test.reset_index()
test

,matnr,zzprncode,zzibrnd,mtart,matkl,TherapeuticClass,PrincipalName,mstae,mstde,mstav,...,idnlf,prat1,prat2,prat3,prat4,prat5,prat6,prat7,prat8,prat9
0,21047156,101276.0,DERMAVEEN SOAP FREE,ZF01,ZCS,All other non-therapeutic products,INOVA PHARMACEUTICALS (S) PTE,LR,20250101,BS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,23046633,101234.0,KENHANCER PLASTER,ZF03,ZMC,Topical antirheumatics,SUN PHARMACEUTICAL SDN BHD,LR,20250101,LR,...,23046633-CCCC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,21210803,101223.0,TECNIS ONE ZCB 06.5D,ZF01,ZMD,Cataract and anticataractogenic preps.,ABBOTT MEDICAL OPTICS,LR,99910101,LR,...,ZCB0000065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,23064902,101369.0,REDOXON KIDS DA CHEW,ZF03,ZCH,Vitamin C combinations,BAYER CO (MALAYSIA) SDN BHD,LR,20250101,LR,...,80450592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,21037570,101223.0,TECNIS ONE ZCB 21.0D,ZF01,ZMD,Cataract and anticataractogenic preps.,ABBOTT MEDICAL OPTICS,LR,20250101,LR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4778,23064645,101369.0,BEPANTHEN FIRST AID,ZF03,ZWM,All other wound healing agents,BAYER CO (MALAYSIA) SDN BHD,LR,20250101,LR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4779,23348769,101228.0,(Z) JANUMET XR 50/,ZF03,ZPE,DPP-IV Inhibitor & Biguanide Antidiab.,MERCK SHARP & DOHME (I.A),LR,20241206,LR,...,1029847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4780,23348901,101219.0,(Z) MABTHERA VIAL,ZF03,ZPE,Other Immunosuppressants,ROCHE (MALAYSIA) SDN BHD,LR,20250201,LR,...,10218134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4781,23348920,101219.0,(Z) ACTEMRA VIAL,ZF03,ZPE,Other Immunosuppressants,ROCHE (MALAYSIA) SDN BHD,LR,22241121,LR,...,10236677,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# word 2 index
nltk_stop_words = stopwords.words ('english')
tokenizer = RegexpTokenizer(r'\w+')


def word2index(corpus):
    idx = 0
    w2i = {}
    all_docs_as_int = []
    doc2index = {}
    j = 0
    for doc in corpus:
        doc_as_int = []
        wordsindoc = tokenizer.tokenize(doc.lower())
        doc2index[doc] = j
        j += 1
        for word in wordsindoc:
            if word not in nltk_stop_words:
                # check if this word exist in dictionary
                if word not in w2i:
                    w2i[word] = idx # dettol as 0
                    idx += 1

                doc_as_int.append(w2i[word])
                
        all_docs_as_int.append(doc_as_int)
    return w2i,all_docs_as_int,doc2index

def index2word(w2i):
    i2w = []
    for k,v in w2i.items():
        i2w.append(k)
    return i2w

In [5]:
#test = test.loc[50:115]
#test = test.reset_index()
#test

In [6]:
test_data = test['Description'][:]

print('test_data\n',test_data)
w2i,docs_as_int,doc2index = word2index(test_data)
i2w = index2word(w2i)
print('word 2 index\n', w2i)
print('docs as int\n',docs_as_int)
print('length of w2i = ', len(w2i))
print('length of doc = ', len(docs_as_int))
print('index2word\n',i2w)
print('doc2index\n',doc2index)

test_data
 0                DERMAVEEN SOAP FREEWASH500ML
1                       KENHANCER PLASTER20'S
2                        TECNIS ONE ZCB 06.5D
3              REDOXON KIDS DA CHEWABLETAB60S
4                        TECNIS ONE ZCB 21.0D
                        ...                  
4778    BEPANTHEN FIRST AIDCREAM30G FREE 3.5G
4779         (Z) JANUMET XR 50/1000 TABS 56'S
4780        (Z) MABTHERA VIAL 500MG/50ML 1 MY
4781         (Z) ACTEMRA VIAL 200MG/10ML 1 MY
4782         FOLTENE ANTI AGING SHAMPOO 200ML
Name: Description, Length: 4783, dtype: object
word 2 index
 {'dermaveen': 0, 'soap': 1, 'freewash500ml': 2, 'kenhancer': 3, 'plaster20': 4, 'tecnis': 5, 'one': 6, 'zcb': 7, '06': 8, '5d': 9, 'redoxon': 10, 'kids': 11, 'da': 12, 'chewabletab60s': 13, '21': 14, '0d': 15, 'synmesh': 16, 'cutter': 17, 'v3': 18, 'fluorocell': 19, 'wdfst2x42ml': 20, 'adalat': 21, 'latabs30mg30': 22, 'zovirax': 23, 'opthoint4': 24, '5gm': 25, 'oxy': 26, 'oil': 27, 'control': 28, 'toner150ml': 29, 'nim

In [7]:
# measure term frequency

# length of term frequency = No Of Documents * No Of Words = 205 * 117
ROWS =    len(docs_as_int) # size of document
COLUMNS = len(w2i)         # size of words
size = ROWS * COLUMNS
print('size of term frequency matrix',size)
term_frequency = np.zeros((ROWS,COLUMNS))
print('shape of term frequence matrix', term_frequency.shape)
def count_vectorizor(doc_as_int):
    for i, doc in enumerate(doc_as_int):
        for j in doc:
            term_frequency[i][j] += 1
count_vectorizor(docs_as_int)
print('term_frequency\n',term_frequency)
print('note term dettol appears in both documents')

size of term frequency matrix 28367973
shape of term frequence matrix (4783, 5931)
term_frequency
 [[1. 1. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]]
note term dettol appears in both documents


In [8]:
# measure inverse document frequency
# idf = log(N/document_frequency)
document_frequency = np.sum(term_frequency>0,axis=0)
print('word appearnce frequency\n',document_frequency)
print('not dettol term appeared in both the test documents at index 0')
idf = np.log(len(docs_as_int)/document_frequency)
print('idf\n',idf)
print('since word dettol appeared in both documents at index 0, its importance for both docs is 0')

word appearnce frequency
 [ 8 16  1 ...  1  1  1]
not dettol term appeared in both the test documents at index 0
idf
 [6.3933817  5.70023452 8.47282324 ... 8.47282324 8.47282324 8.47282324]
since word dettol appeared in both documents at index 0, its importance for both docs is 0


In [9]:
# measure tfidf
tfidf = term_frequency * idf
print('tfidf\n',tfidf)

tfidf
 [[6.3933817  5.70023452 8.47282324 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         8.47282324 0.        ]
 [0.         0.         0.         ... 0.         0.         8.47282324]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [10]:
from scipy import sparse
tfidf_vector = sparse.csr_matrix(tfidf)
print(tfidf_vector.shape)
print(type(tfidf_vector))
#idx = doc2index['DETTOL BS NOURISHING120G3+1']
idx = doc2index['DETTOL TALCUM MENTHOL(NEW)150G']
print(idx)
query = tfidf_vector[idx]
query

(4783, 5931)
<class 'scipy.sparse.csr.csr_matrix'>
4758


<1x5931 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [11]:
scores = cosine_similarity(query,tfidf_vector)
scores = scores.flatten()
print(scores.shape)
print(tfidf_vector.shape)
print(scores)
(-scores).argsort()

(4783,)
(4783, 5931)
[0. 0. 0. ... 0. 0. 0.]


array([4758, 3349,  554, ..., 1619, 1636, 4782], dtype=int64)

In [12]:
recommendation = (-scores).argsort()[0:10]
print(recommendation)
test['Description'].iloc[recommendation]

[4758 3349  554 1603 3866  614 3727 4674 4285  820]


4758     DETTOL TALCUM MENTHOL(NEW)150G
3349    DETTOL TALCUM ORIGINAL(NEW)150G
554                    DETTOL CREAM 30G
1603          DETTOL HANDWASH500MLFRESH
3866      DETTOL ANTIBACTERIALWIPES50'S
614       DETTOL ANTIBACTERIALWIPES10'S
3727       DETTOL HANDWASH250MLSKINCARE
4674      DETTOL HANDWASH250MLSENSETIVE
4285       DETTOL HANDWASH500MLORIGINAL
820        DETTOL HANDWASH500MLSKINCARE
Name: Description, dtype: object

In [13]:
def recommend(title):
    # find the idx of the movie
    idx = doc2index[title]
    query = tfidf_vector[idx]
    scores = cosine_similarity(query,tfidf_vector)
    scores = scores.flatten()
    recommendation = (-scores).argsort()[0:10]
    return test['Description'].iloc[recommendation]

In [14]:
recommend('DETTOL TALCUM MENTHOL(NEW)150G')

4758     DETTOL TALCUM MENTHOL(NEW)150G
3349    DETTOL TALCUM ORIGINAL(NEW)150G
554                    DETTOL CREAM 30G
1603          DETTOL HANDWASH500MLFRESH
3866      DETTOL ANTIBACTERIALWIPES50'S
614       DETTOL ANTIBACTERIALWIPES10'S
3727       DETTOL HANDWASH250MLSKINCARE
4674      DETTOL HANDWASH250MLSENSETIVE
4285       DETTOL HANDWASH500MLORIGINAL
820        DETTOL HANDWASH500MLSKINCARE
Name: Description, dtype: object

In [15]:
recommend('DERMADRATE DRY SKINTREATMENT CREAM 500G')

2130    DERMADRATE DRY SKINTREATMENT CREAM 500G
194      DERMADRATE DRY SKINTREATMENT CREAM 50G
3434                 VEET CREAM 100ML + 25G DRY
3055              VEET CREAM 100ML DRY(MISSONI)
1280                    JB POWDER BLOSSOMS 500G
2244                *JB POWDER MILK + RICE 500G
215                             VEET RASERA DRY
1045                   VEET IN SHOWER 150ML DRY
2362                MUSCLE BUILDER VAN-PWD 500G
792                MUSCLE BUILDER CHOC-PWD 500G
Name: Description, dtype: object

In [16]:
recommend('DERMAVEEN INTENSIVEHAND CREAM100ML')

4140     DERMAVEEN INTENSIVEHAND CREAM100ML
685                 HAIR REVIVAL CREAM100ML
2492                   HAIR LOSS CREAM100ML
3460      DERMAVEEN BATH SACHETS50GX5SACHET
0              DERMAVEEN SOAP FREEWASH500ML
472            DERMAVEEN SOAP FREEWASH250ML
4060    DERMAVEEN MOISTURISING LOTION 250ML
613     DERMAVEEN MOISTURISING LOTION 500ML
4088         DERMAVEEN OATMEAL SHAMPOO500ML
2877      DERMAVEEN CONDITIONER BOTTLE250ML
Name: Description, dtype: object

In [17]:
recommend('FOLTENE ANTI AGING SHAMPOO 200ML')

4782            FOLTENE ANTI AGING SHAMPOO 200ML
3424       FOLTENE STRENGTHENING M SHAMPOO 200ML
1048             FOLTENE MEN'S SHAMPOO 200ML 1'S
2668       FOLTENE DERMOPROTECTIVE SHAMPOO 200ML
4305        FOLTENE ANTI DANDRUFF DRY/OILY 200ML
4698      FOLTENE STRENGTHENING WM SHAMPOO 200ML
3815        ANTI-HAIR FALL SHAMPOO (400ML+200ML)
551      FOLTENE SEBUM NORMALIZING SHAMPOO 200ML
4077    ANTI-DANDRUFF SHAMPOO GENTLE CLEAN 200ML
4573       TDF ANTI-AGING CLEANNG GEL3.5OZ/100ML
Name: Description, dtype: object

In [18]:
recommend('JOHNSON\'S LOTION SOAP 100GMX3')

11       JOHNSON'S LOTION SOAP 100GMX3
1959        JOHNSON'S BABY SOAP3X100GM
651     JOHNSON BABY GOLD SHAMPOO200ML
0         DERMAVEEN SOAP FREEWASH500ML
472       DERMAVEEN SOAP FREEWASH250ML
1133    JOHNSON BABY GOLD SHAMPOO100ML
4245     JOHNSON BABY COND.S'POO 100ML
3627     JOHNSON BABY COND.S'POO 200ML
787               JB LOTION 100ML-PINK
4099              JB LOTION 200ML-PINK
Name: Description, dtype: object

In [19]:
idx = doc2index['JOHNSON\'S LOTION SOAP 100GMX3']
print(idx)
query = tfidf_vector[idx]
query


11


<1x5931 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [20]:
r = query.toarray().flatten()
k = 0
for i in r:
    k += 1
    if i > 0:
        print('idx word = ', i2w[k-1], ' value ' , i, 'at index ',k)

idx word =  soap  value  5.700234521440516 at index  2
idx word =  johnson  value  5.700234521440516 at index  32
idx word =  lotion  value  4.976315682213817 at index  33
idx word =  100gmx3  value  8.472823243680297 at index  34


In [21]:
type(query.toarray())

numpy.ndarray

In [44]:
def word2_product_recommendation(word,tfidf_vector,w2i,weight,data,test):
    mock_query = np.zeros((1,tfidf_vector.shape[1]))
    idx = w2i[word]
    mock_query[0][idx] = weight
    scores = cosine_similarity(mock_vector,tfidf_vector)
    recommendation = (-scores).argsort()[0:10]
    return data['Description'].iloc[recommendation]

In [46]:
word = 'synmesh'
word2_product_recommendation(word,tfidf_vector,w2i,10,test)

ValueError: Wrong number of dimensions. values.ndim > ndim [2 > 1]

In [32]:
# find the index of the word to search for

idx = w2i[word]
query = tfidf_vector[0]
q = query.toarray().flatten()
print(q.shape)
for x in range(0, q.shape[0]):
     q[x] = 0
print('word at ', idx, ' is ' + i2w[idx])
q[idx] = 10
mock_vector = sparse.csr_matrix(q)
print(mock_vector.shape)
print(type(mock_vector))
scores = cosine_similarity(mock_vector,tfidf_vector)
scores = scores.flatten()
print(scores.shape)
print(tfidf_vector.shape)
print(scores)
(-scores).argsort()
recommendation = (-scores).argsort()[0:10]
print(recommendation)
test['Description'].iloc[recommendation]

(5931,)
word at  16  is synmesh
(1, 5931)
<class 'scipy.sparse.csr.csr_matrix'>
(4783,)
(4783, 5931)
[0. 0. 0. ... 0. 0. 0.]
[2193 2619 1421 4235 4390  902 2883    5 2813 2958]


2193               SynMesh Cutter
2619       Vario Case for SynMesh
1421       Vario Case for SynMesh
4235       Vario Case for SynMesh
4390       Vario Case for SynMesh
902        Vario Case for SynMesh
2883            SynMesh Cutter V2
5               SynMesh Cutter V3
2813    Vario Case for SynMesh V2
2958    Vario Case for SynMesh V3
Name: Description, dtype: object

In [23]:
print(i2w[3])

kenhancer


In [24]:
tfidf[0][:4]

array([6.3933817 , 5.70023452, 8.47282324, 0.        ])

In [30]:
i2w

['dermaveen',
 'soap',
 'freewash500ml',
 'kenhancer',
 'plaster20',
 'tecnis',
 'one',
 'zcb',
 '06',
 '5d',
 'redoxon',
 'kids',
 'da',
 'chewabletab60s',
 '21',
 '0d',
 'synmesh',
 'cutter',
 'v3',
 'fluorocell',
 'wdfst2x42ml',
 'adalat',
 'latabs30mg30',
 'zovirax',
 'opthoint4',
 '5gm',
 'oxy',
 'oil',
 'control',
 'toner150ml',
 'nimotoptabs30mg30',
 'johnson',
 'lotion',
 '100gmx3',
 'magnesium',
 'amino',
 'acidchelate',
 '75060s',
 'lamisiltab250mg3x14',
 'wellbutrin',
 'sr150mg60s',
 'zaditen',
 'syrupbot1mg',
 '5ml200ml',
 'zocortabs10mg120',
 'tubinette',
 '20m',
 'roll',
 '01',
 'ux',
 'ii',
 'search',
 'sed',
 'st29mlx',
 '1',
 'uriage',
 'depiderm',
 'soinintensif',
 'f30ml',
 'cu',
 'zn',
 'gel',
 'nettoyantf',
 '200ml',
 'nl',
 'l',
 'eau',
 'demaquillantepnm',
 '250ml',
 'urgosoft',
 '4cmx1m1',
 'pregnancy',
 'formulacap60',
 'syoss',
 'anti',
 'dandruffctrl',
 'c',
 'tioner190ml',
 'stieva',
 'cream',
 '0',
 '05',
 '25g',
 'stimulen',
 'diabetic',
 'care',
 '2oz',
 